In [11]:
from google.colab import drive
drive.mount("/content/gdrive/")
import os
os.chdir("/content/gdrive/My Drive/MultimodalEmotion/MODULE2")
!ls

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
angry.jpeg		 fer_model.h5			      model.ipynb
data			 haarcascade_frontalface_default.xml  neutral.jpeg
emotion_detection.ipynb  happy.jpeg			      sad.jpeg
fear.jpg		 image_emotion.npy		      test_image_data.npy


In [12]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
# Set the dimensions of the input images
img_width, img_height = 48, 48

# Set the directories for the training and validation datasets
train_data_dir = '/data/train'
validation_data_dir = '/data/test'

# Set the number of training and validation samples
nb_train_samples = 28709
nb_validation_samples = 3589

# Set the batch size and number of epochs
batch_size = 64
epochs = 50

# Set the number of classes (emotions)
num_classes = 7

In [14]:
np.save("test_image_data.npy", validation_data_dir)

In [15]:
# Create the model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])

# Create the data generators for the training and validation datasets
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, color_mode='grayscale', target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory(validation_data_dir, color_mode='grayscale', target_size=(img_width, img_height), batch_size=batch_size, class_mode='categorical')

FileNotFoundError: [Errno 2] No such file or directory: '/data/train'

In [ ]:
# Train the model
model.fit_generator(train_generator, steps_per_epoch=nb_train_samples//batch_size, epochs=epochs, validation_data=validation_generator, validation_steps=nb_validation_samples//batch_size)

In [ ]:
print(model.summary())

In [ ]:
# Save the model
model.save('fer_model.h5')